# Machine learning on wine

**Topics:** Text analysis, linear regression, logistic regression, text analysis, classification

**Datasets**

- **wine-reviews.csv** Wine reviews scraped from https://www.winemag.com/
- **Data dictionary:** just go [here](https://www.winemag.com/buying-guide/tenuta-dellornellaia-2007-masseto-merlot-toscana/) and look at the page

## The background

You work in the **worst newsroom in the world**, and you've had a hard few weeks at work - a couple stories killed, a few scoops stolen out from under you. It's not going well.

And because things just can't get any worse: your boss shows up, carrying a huge binder. She slams it down on your desk.

"You know some machine learning stuff, right?"

You say "no," but she isn't listening. She's giving you an assignment, the _worst assignment_...

> Machine learning is the new maps. Let's get some hits!
>
> **Do some machine learning on this stuff.**

"This stuff" is wine reviews.

## A tiny, meagre bit of help

You have a dataset. It has some stuff in it:

* **Numbers:**
    - Year published
    - Alcohol percentage
    - Price
    - Score
    - Bottle size
* **Categories:**
    - Red vs white
    - Different countries
    - Importer
    - Designation
    - Taster
    - Variety
    - Winery
* **Free text:**
    - Wine description

# Cleaning up your data

Many of these pieces - the alcohol, the year produced, the bottle size, the country the wine is from - aren't in a format you can use. Convert the ones to numbers that are numbers, and extract the others from the appropriate strings.

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_csv("wine-reviews.csv")
df.head(10)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating
0,https://www.winemag.com/buying-guide/artadi-2011-vinas-gain-tempranillo-rioja/,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black plum and coconut filter into a round, fluffy palate that's friendly and pure but not very dense or structured. Baked flavors of molasses and gamy berry ...",Michael Schachner,"$25, Buy Now",Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5%,750 ml,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review]
1,https://www.winemag.com/buying-guide/adelsheim-2012-stoller-vineyard-chardonnay-willamette-valley-dundee-hills/,90.0,Adelsheim 2012 Stoller Vineyard Chardonnay (Dundee Hills),"A tiny production wine, this is rich, tart and vividly fruity. The generous mix of citrus, apple and peach fruit is augmented by barrel fermentation flavors of toasted hazelnuts, caramel and bakin...",Paul Gregutt,"$65, Buy Now",Stoller Vineyard,Chardonnay,"Dundee Hills, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
2,https://www.winemag.com/buying-guide/adelsheim-2013-ribbon-springs-vineyard-other-white-auxerrois-willamette-valley-ribbon-ridge/,90.0,Adelsheim 2013 Ribbon Springs Vineyard Auxerrois (Ribbon Ridge),"This is another fine vintage for this rare wine. It's loaded with cool climate, mineral-laced scents of grapefruit, kiwi and melon. A whiff of fennel adds further interest. Super refreshing and a ...",Paul Gregutt,"$25, Buy Now",Ribbon Springs Vineyard,"Auxerrois, Other White","Ribbon Ridge, Willamette Valley, Oregon, US",Adelsheim,13.5%,750 ml,White,NaN,12/1/2014,Not rated yet [Add Your Review]
3,https://www.winemag.com/buying-guide/jcb-2011-no-11-pinot-noir-sonoma-coast/,90.0,JCB 2011 No. 11 Pinot Noir (Sonoma Coast),"Light in color and lilting floral aromas of rose, this is an inviting cool-climate Pinot Noir swirling in equal parts strawberry and spice, subtle and sophisticated.",Virginie Boone,"$65, Buy Now",No. 11,Pinot Noir,"Sonoma Coast, Sonoma, California, US",JCB,13%,750 ml,Red,NaN,12/1/2014,Not rated yet [Add Your Review]
4,https://www.winemag.com/buying-guide/pazo-pondal-2013-albarino-rias-baixas/,90.0,Pazo Pondal 2013 Albariño (Rías Baixas),"Alluring, inviting aromas of white flowers, melon and peach are pure as stream water. This feels round and juicy, with flavors of green herbs, lettuce, lime and orange. Tangerine notes carry the f...",Michael Schachner,"$17, Buy Now",NaN,Albariño,"Rías Baixas, Galicia, Spain",Pazo Pondal,13%,750 ml,White,Vinaio Imports,12/1/2014,Not rated yet [Add Your Review]
5,https://www.winemag.com/buying-guide/mumm-napa-2008-dvx-rose-sparkling-napa-valley/,90.0,Mumm Napa 2008 DVX Rosé Sparkling (Napa Valley),"Pretty peach in color, this 50-50 sparkling blend of Pinot Noir and Chardonnay, with a small percentage of the Pinot added to the blend before secondary fermentation, is a light, high-toned, flint...",Virginie Boone,"$70, Buy Now",DVX Rosé,"Sparkling Blend, Sparkling","Napa Valley, Napa, California, US",Mumm Napa,12.5%,750 ml,Sparkling,NaN,12/1/2014,Not rated yet [Add Your Review]
6,https://www.winemag.com/buying-guide/nuiton-beaunoy-2011-clos-chapitre-premier-cru-pinot-noir-gevrey-chambertin/,90.0,Nuiton-Beaunoy 2011 Clos du Chapitre Premier Cru (Gevrey-Chambertin),"The two-acre Clos du Chapitre vineyard is in the center of the Gevrey-Chambertin village. The wine is rich, full-bodied, full of red fruits, dark cherries and a rounded, perfumed character. It's p...",Roger Voss,"N/A, Buy Now",Clos du Chapitre Premier Cru,Pinot Noir,"Gevrey-Chambertin, Burgundy, France",Nuiton-Beaunoy,13%,750 ml,Red,"Fruit of the Vines, Inc",12/1/2014,Not rated yet [Add Your Review]
7,https://www.winemag.com/buying-guide/trapiche-2012-broquel-cabernet-sauvignon-mendoza/,90.0,Trapiche 2012 Broquel Cabernet Sauvign

In [3]:
df.shape

(42295, 16)

In [4]:
# cleaning price column
df.price = df.price.str.replace("$", "")

In [5]:
df.price = df.price.str.replace("Buy Now", "")

In [6]:
df.price = df.price.replace(',','', regex=True)

In [7]:
# cleaning alcohol column
df.alcohol = df.alcohol.replace('%','', regex=True)

In [8]:
# cleaning bottle size column
df['bottle size'] = df['bottle size'].str.replace('ml','')

In [9]:
# cleaning country column
df['country']= df.appellation.str.rsplit(',').str[-1]

In [10]:
# cleaning year column
df['year'] = df['date published'].str[-4:]

In [11]:
df['year'] = df['year'].astype(float)

In [12]:
df['year'] = 2021 - df['year']

In [13]:
#df.isna().sum()

In [14]:
df.head(1)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,country,year
0,https://www.winemag.com/buying-guide/artadi-2011-vinas-gain-tempranillo-rioja/,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black plum and coconut filter into a round, fluffy palate that's friendly and pure but not very dense or structured. Baked flavors of molasses and gamy berry ...",Michael Schachner,25,Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5,750,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review],Spain,7.0


In [15]:
# QUESTION
# I CAN'T DROP N/A

df.dropna(how='any', inplace=True)

In [16]:
df.shape

(13318, 18)

In [17]:
df.head(10)

,url,wine_points,wine_name,wine_desc,taster,price,designation,variety,appellation,winery,alcohol,bottle size,category,importer,date published,user avg rating,country,year
0,https://www.winemag.com/buying-guide/artadi-2011-vinas-gain-tempranillo-rioja/,90.0,Artadi 2011 Viñas de Gain (Rioja),"Inky, minerally aromas of blackberry, black plum and coconut filter into a round, fluffy palate that's friendly and pure but not very dense or structured. Baked flavors of molasses and gamy berry ...",Michael Schachner,25,Viñas de Gain,Tempranillo,"Rioja, Northern Spain, Spain",Artadi,14.5,750,Red,Folio Fine Wine Partners,12/1/2014,Not rated yet [Add Your Review],Spain,7.0
6,https://www.winemag.com/buying-guide/nuiton-beaunoy-2011-clos-chapitre-premier-cru-pinot-noir-gevrey-chambertin/,90.0,Nuiton-Beaunoy 2011 Clos du Chapitre Premier Cru (Gevrey-Chambertin),"The two-acre Clos du Chapitre vineyard is in the center of the Gevrey-Chambertin village. The wine is rich, full-bodied, full of red fruits, dark cherries and a rounded, perfumed character. It's p...",Roger Voss,N/A,Clos du Chapitre Premier Cru,Pinot Noir,"Gevrey-Chambertin, Burgundy, France",Nuiton-Beaunoy,13,750,Red,"Fruit of the Vines, Inc",12/1/2014,Not rated yet [Add Your Review],France,7.0
7,https://www.winemag.com/buying-guide/trapiche-2012-broquel-cabernet-sauvignon-mendoza/,90.0,Trapiche 2012 Broquel Cabernet Sauvignon (Mendoza),"Spice, licorice and herbal notes complement red-fruit aromas, while the palate offers plenty of structure and tannic grab. There's an avalanche of blackberry, cassis, fig, chocolate and herbal fla...",Michael Schachner,15,Broquel,Cabernet Sauvignon,"Mendoza, Mendoza Province, Argentina",Trapiche,14,750,Red,The Wine Group,12/1/2014,Not rated yet [Add Your Review],Argentina,7.0
10,https://www.winemag.com/buying-guide/peter-nicolay-2012-erdener-treppchen-feinherb-riesling-mosel/,90.0,Peter Nicolay 2012 Erdener Treppchen Feinherb Riesling (Mosel),"Honey-kissed peaches waft from this ripe, rich off-dry Riesling full of juicy, luscious stone fruit and citrus. It's quaffable and thirst quenching, yet refreshingly brisk with a lingering shower ...",Anna Lee C. Iijima,20,Erdener Treppchen Feinherb,Riesling,"Mosel, Germany",Peter Nicolay,12,750,White,Saranty Imports,12/1/2014,Not rated yet [Add Your Review],Germany,7.0
11,https://www.winemag.com/buying-guide/pol-roger-nv-reserve-brut-champagne-202047/,90.0,Pol Roger NV Réserve Brut (Champagne),"Full and ripe, this offers balance between rich white fruits and a smooth texture that is cut with acidity. The wine is in great harmony with itself, ready to drink while also allowing the possibi...",Roger Voss,64,Réserve Brut,"Champagne Blend, Sparkling","Champagne, Champagne, France",Pol Roger,12.5,750,Sparkling,"Frederick Wildman & Sons, Ltd",12/1/2014,Not rated yet [Add Your Review],France,7.0
12,https://www.winemag.com/buying-guide/quinta-pacos-2013-casa-capitao-mor-albarino-alvarinho-vinho-verde/,90.0,Quinta de Paços 2013 Casa do Capitão-mor Alvarinho (Vinho Verde),"Named after the 13th-century house at the center of the estate, this is a rich, full-bodied wine. With citrus flavors at its heart, it also shows a texture that gives minerality from the granite s...",Roger Voss,18,Casa do Capitão-mor,"Alvarinho, Albariño","Vinho Verde, Portugal",Quinta de Paços,13,750,White,Aidil Wines/Old World Import,12/1/2014,Not rated yet [Add Your Review],Portugal,7.0
13,https://www.winemag.com/buying-guide/quinta-noval-nv-10-years-old-tawny-port-blend-port-port/,90.0,Quinta do Noval NV 10-Years-Old Tawny (Port),"This dry, balanced aged tawny is both fruity and showing good aging characters. Red fruits are cut with acidity and shaped by the old wood and spice flavors. It has a delicious, ripe aftertaste.",Roger Voss,30,10-Years-Old Tawny,"Port, Port Blend","Port, Portugal",Quinta do Noval,20,750,Port/Sherry,Vintus LLC,12/1/2014,Not rated yet [Add Your Review],Portugal,7.0
15,https://www.winemag.com/buying-guide/maso-martis-2007-b

In [18]:
df.dtypes

url                 object
wine_points        float64
wine_name           object
wine_desc           object
taster              object
price               object
designation         object
variety             object
appellation         object
winery              object
alcohol             object
bottle size         object
category            object
importer            object
date published      object
user avg rating     object
country             object
year               float64
dtype: object

## What might be interesting in this dataset?

Maybe start out playing around _without_ machine learning. Here are some thoughts to get you started:

* I've heard that since the 90's wine has gone through [Parkerization](https://www.estatewinebrokers.com/blog/the-parkerization-of-wine-in-the-1990s-and-beyond/), an increase in production of high-alcohol, fruity red wines thanks to the influence of wine critic Robert Parker.
* Red and white wines taste different, obviously, but people always use [goofy words to describe them](https://winefolly.com/tutorial/40-wine-descriptions/)
* Once upon a time in 1976 [California wines proved themselves against France](https://en.wikipedia.org/wiki/Judgment_of_Paris_(wine)) and France got very angry about it

In [19]:
# finding out correlation between wine_points and price

In [20]:
# finding out correlation between wine_points and wine desc

In [21]:
# finding out correlation between wine_points and variety

In [22]:
# finding out correlation between wine desc and category

In [23]:
# finding out correlation between variety and country

## But machine learning?

Well, you can usually break machine learning down into a few different things. These aren't necessarily perfect ways of categorizing things, but eh, close enough.

* **Predicting a number**
    - Linear regression
    - For example, how does a change in unemployment translate into a change in life expectancy?
* **Predicting a category** (aka classification)
    - Lots of algos options: logistic regression, random forest, etc
    - For example, predicting cuisines based on ingredients
* **Seeing what influences a numeric outcome**
    - Linear regression since the output is a number
    - For example, minority and poverty status on test scores 
* **Seeing what influences a categorical outcome**
    - Logistic regression since the output is a category
    - Race and car speed for if you get a waring vs ticket
    - Wet/dry pavement and car weight if you survive or not in a car crash)

We have numbers, we have categories, we have all sorts of stuff. **What are some ways we can mash them together and use machine learning?**

### Brainstorm some ideas

Use the categories above to try to come up with some ideas. Be sure to scroll up where I break down categories vs numbers vs text!

**I'll give you one idea for free:** if you don't have any ideas, start off by creating a classifier that determines whether a wine is white or red based on the wine's description.

In [24]:
# Linear regression - how does the price affect wine ratings?

In [25]:
# Predicting a category - predicting wine category based on wine description 

In [26]:
# Seeing what influences a numeric outcome - 
# country and variety on ratings

You can also go to https://library.columbia.edu and see if you can find some academic papers about wine. I'm sure they'll inspire you! (and they might even have some ML ideas in them you can steal, too)

# Implement 2 of your machine learning ideas

## IDEA 1: Linear regression - how does the year affect wine ratings?

In [27]:
df_LR = df[['year', 'wine_points']]

In [28]:
df_LR.head()

,year,wine_points
0,7.0,90.0
6,7.0,90.0
7,7.0,90.0
10,7.0,90.0
11,7.0,90.0


In [29]:
df_LR.isna().sum()

year           0
wine_points    0
dtype: int64

In [30]:
df_LR.dtypes

year           float64
wine_points    float64
dtype: object

In [33]:
df_LR.year = df_LR.year.astype(float)

/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [34]:
import statsmodels.api as sm

# What effect does the year have on ratings?
X = df_LR[['year']]
y = df_LR.wine_points

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

In [35]:
results.params

const    90.725928
year     -0.210720
dtype: float64

In [36]:
results.summary()

# ANALYSIS
# For every "year" a wine ages, it gets 0.15 less "points"
# I'm guessing that's b/c older wine tends to be more expencive
# I can't get python to drop N/A wine prices...

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            wine_points   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     975.3
Date:                Thu, 01 Apr 2021   Prob (F-statistic):          1.08e-206
Time:                        01:06:11   Log-Likelihood:                -34334.
No. Observations:               13318   AIC:                         6.867e+04
Df Residuals:                   13316   BIC:                         6.869e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         90.7259      0.064   1412.621      0.000      90.600      90.852
year          -0.2107      0.007    -31.230      0.000      -0.224      -0.197
==============================================================================
Omnibus:                      220.820   Durbin-Watson:                   0.381
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.689
Skew:                          -0.262   Prob(JB):                     3.57e-45
Kurtosis:                       2.693   Cond. No.                         22.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
import statsmodels.formula.api as smf
import numpy as np

model = smf.ols(formula='wine_points ~ np.multiply(year, 100)', data=df_LR)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            wine_points   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     975.3
Date:                Thu, 01 Apr 2021   Prob (F-statistic):          1.08e-206
Time:                        01:06:11   Log-Likelihood:                -34334.
No. Observations:               13318   AIC:                         6.867e+04
Df Residuals:                   13316   BIC:                         6.869e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 90.7259      0.064   1412.621      0.000      90.600      90.852
np.multiply(year, 100)    -0.0021   6.75e-05    -31.230      0.000      -0.002      -0.002
==============================================================================
Omnibus:                      220.820   Durbin-Watson:                   0.381
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.689
Skew:                          -0.262   Prob(JB):                     3.57e-45
Kurtosis:                       2.693   Cond. No.                     2.21e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.21e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## IDEA 2: Predicting a category - predicting wine category based on wine description 

In [39]:
df_P = df[['wine_desc', 'category']]

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

# Make a vectorizer
vectorizer = CountVectorizer()

# Learn and count the words in df.content
matrix = vectorizer.fit_transform(df_P.wine_desc)

# Convert the matrix of counts to a dataframe
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names())

In [41]:
words_df.head(1)

,000,01,02,02s,04,05,06,06s,07,08,09,10,100,1000,11,110,114,117,12,125,1290,12th,13,13th,14,14th,15,150,1500,150th,155,15th,16,1610,1667,166th,1674,16th,17,1756,1763,1789,1791,17th,18,180,1800,1806,1850s,1855,...,yours,yourself,youth,youthful,youthfully,ysios,yummy,yung,yup,yuzu,yves,zamora,zantho,zap,zaps,zealand,zellenberg,zeller,zelma,zero,zest,zestier,zestiness,zesty,zibibbo,zierfandler,zimmermann,zin,zinfandel,zing,zinger,zings,zingy,zip,zippiness,zippy,zips,zone,zuccardi,zull,zuri,zweigelt,zwerithaler,zwiegelt,zédé,zéro,élevage,émilion,épernay,ürziger
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
df_P['is_red'] = (df_P['category'] == 'Red').astype(int)
df_P.head()

<ipython-input-43-4a818d8e0ebd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_P['is_red'] = (df_P['category'] == 'Red').astype(int)


,wine_desc,category,is_red
0,"Inky, minerally aromas of blackberry, black plum and coconut filter into a round, fluffy palate that's friendly and pure but not very dense or structured. Baked flavors of molasses and gamy berry ...",Red,1
6,"The two-acre Clos du Chapitre vineyard is in the center of the Gevrey-Chambertin village. The wine is rich, full-bodied, full of red fruits, dark cherries and a rounded, perfumed character. It's p...",Red,1
7,"Spice, licorice and herbal notes complement red-fruit aromas, while the palate offers plenty of structure and tannic grab. There's an avalanche of blackberry, cassis, fig, chocolate and herbal fla...",Red,1
10,"Honey-kissed peaches waft from this ripe, rich off-dry Riesling full of juicy, luscious stone fruit and citrus. It's quaffable and thirst quenching, yet refreshingly brisk with a lingering shower ...",White,0
11,"Full and ripe, this offers balance between rich white fruits and a smooth texture that is cut with acidity. The wine is in great harmony with itself, ready to drink while also allowing the possibi...",Sparkling,0


In [45]:
X = words_df
y = df_P.is_red

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
from sklearn.tree import DecisionTreeClassifier 
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [48]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not red', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not red,Predicted red
Is not red,1586,76
Is red,273,1395


In [49]:
from sklearn.svm import LinearSVC 
clf = LinearSVC(max_iter=10000)
clf.fit(X_train, y_train)

LinearSVC(max_iter=10000)

In [50]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not red', 'red'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not red,Predicted red
Is not red,1597,65
Is red,78,1590


In [51]:
import eli5

eli5.show_weights(clf, feature_names=vectorizer.get_feature_names())

In [53]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_P['category_label'] = le.fit_transform(df_P.category)
df_P.head()

<ipython-input-53-0b9c3c6f3671>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_P['category_label'] = le.fit_transform(df_P.category)


,wine_desc,category,is_red,category_label
0,"Inky, minerally aromas of blackberry, black plum and coconut filter into a round, fluffy palate that's friendly and pure but not very dense or structured. Baked flavors of molasses and gamy berry ...",Red,1,3
6,"The two-acre Clos du Chapitre vineyard is in the center of the Gevrey-Chambertin village. The wine is rich, full-bodied, full of red fruits, dark cherries and a rounded, perfumed character. It's p...",Red,1,3
7,"Spice, licorice and herbal notes complement red-fruit aromas, while the palate offers plenty of structure and tannic grab. There's an avalanche of blackberry, cassis, fig, chocolate and herbal fla...",Red,1,3
10,"Honey-kissed peaches waft from this ripe, rich off-dry Riesling full of juicy, luscious stone fruit and citrus. It's quaffable and thirst quenching, yet refreshingly brisk with a lingering shower ...",White,0,6
11,"Full and ripe, this offers balance between rich white fruits and a smooth texture that is cut with acidity. The wine is in great harmony with itself, ready to drink while also allowing the possibi...",Sparkling,0,5


In [54]:
X = words_df
y = df_P.category_label

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [56]:
from sklearn.svm import LinearSVC 
clf = LinearSVC(max_iter=10000)
clf.fit(X_train, y_train)

LinearSVC(max_iter=10000)

In [57]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(le.classes_)
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted Dessert,Predicted Fortified,Predicted Port/Sherry,Predicted Red,Predicted Rose,Predicted Sparkling,Predicted White
Is Dessert,34,0,1,6,1,1,24
Is Fortified,1,0,1,0,1,0,1
Is Port/Sherry,2,0,37,17,0,3,2
Is Red,2,0,2,1621,11,6,25
Is Rose,1,0,2,27,84,11,16
Is Sparkling,3,0,0,17,14,277,71
Is White,9,1,1,12,12,42,931


In [58]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(le.classes_)
scores = pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)
scores.style.background_gradient(cmap='YlGnBu')

# ANALYSIS
# Correct predictions for RED and WHITE wine are more than 90%.
# This means that the two types of wines have the most distinctive flavor.
# I can make suggestions based on the result: 
# if you're looking for red wine and the store is out of it, go for port/sherry.
# if you're looking for white wine and the store is out of it, go for dessert.

,Predicted Dessert,Predicted Fortified,Predicted Port/Sherry,Predicted Red,Predicted Rose,Predicted Sparkling,Predicted White
Is Dessert,0.507463,0.000000,0.014925,0.089552,0.014925,0.014925,0.358209
Is Fortified,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.250000
Is Port/Sherry,0.032787,0.000000,0.606557,0.278689,0.000000,0.049180,0.032787
Is Red,0.001200,0.000000,0.001200,0.972406,0.006599,0.003599,0.014997
Is Rose,0.007092,0.000000,0.014184,0.191489,0.595745,0.078014,0.113475
Is Sparkling,0.007853,0.000000,0.000000,0.044503,0.036649,0.725131,0.185864
Is White,0.008929,0.000992,0.000992,0.011905,0.011905,0.041667,0.923611


In [ ]:
# QUESTION - Why do we need machine learning-based prediction when we already know the wine category?